In [206]:
import sklearn
import csv
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy

In [188]:
#spectra = pd.read_csv('/Users/sergiomarconi/OneDrive/Projects/OSBS/inputs/Spectra/CrownPix_norm_3N_pct.csv')
spectra = pd.read_csv('/Users/sergiomarconi/OneDrive/Projects/OSBS/inputs/Bootstrap_2/onePix1Crown_N_pct94.csv')

traits = pd.read_csv('/Users/sergiomarconi/OneDrive/Projects/OSBS/inputs/Spectra/CrownTraits.csv')
print('rows spectra:', len(spectra), '\n rows traits:',len(traits))

rows spectra: 76 
 rows traits: 76


In [189]:
#traits = traits.drop(traits.index[len(traits)-1]) 
average_spectra = spectra.groupby(['pixel_crownID'], as_index = False).mean()
#sd_spectra = spectra.groupby(['pixel_crownID'], as_index = False).std()
#sd_spectra['pixel_crownID'] = average_spectra['pixel_crownID'] 
scaled_spectra = average_spectra - average_spectra.mean()
scaled_spectra['pixel_crownID'] = average_spectra['pixel_crownID'] 

In [190]:
scaled_spectra = scaled_spectra.dropna(axis=1)

In [191]:
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel

Ok, here comes the most intriguing and customizable part. The choice and built of kernel. For Now, I am trying the one proposed by the tutorial. It presents:
* Matèrn component (Matern):
* amplitude factor (ConstantKernel):
* observation noise (WhiteKernel):

In [192]:
kernel = ConstantKernel() + Matern(length_scale=2, nu=3/2) + WhiteKernel(noise_level=1)

In [193]:
X = scaled_spectra.drop("pixel_crownID", 1)
Y = traits['N_pct']
gp = gaussian_process.GaussianProcessRegressor(kernel=kernel)
print('rows spectra:', len(X.index), '\n rows traits:',len(Y.index))

rows spectra: 76 
 rows traits: 76


In [194]:
gp.fit(X,Y)
gp.kernel_
gp.score(X,Y)

0.99999999938925033

In [195]:
#train test split taken for the same dataset in R
indx = [1,11,2,58,3,27,4,35,5,37,6,34,45,13,41,46,21,36,38,49,70,44,69,48,71,53,66,76,57,64]
indx =[x-1 for x in indx]
test_spectra = scaled_spectra.ix[indx]
train_spectra = scaled_spectra.drop(scaled_spectra.index[indx])
print('test spectra:', len(test_spectra.index), '\n train spectra:',len(train_spectra.index))

test spectra: 30 
 train spectra: 46


In [196]:
test_traits = traits.ix[indx]
train_traits = traits.drop(traits.index[indx])
print('test spectra:', len(test_traits.index), '\n train spectra:',len(train_traits.index))

test spectra: 30 
 train spectra: 46


In [197]:
X = train_spectra.drop("pixel_crownID", 1)
Y = train_traits['N_pct']
gp = gaussian_process.GaussianProcessRegressor(kernel=kernel)
print('rows spectra:', len(X.index), '\n rows traits:',len(Y.index))

rows spectra: 46 
 rows traits: 46


In [198]:
gp.fit(X,Y)
gp.kernel_
gp.score(X,Y)

0.43622620755912023

In [202]:
x_test = test_spectra.drop("pixel_crownID", 1)
y_pred, sigma = gp.predict(x_test, return_std=True)


In [203]:
def rsquared(x, y):
    """ Return R^2 where x and y are array-like."""

    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    return r_value**2

In [207]:
print(rsquared(y_pred,test_traits['N_pct']))


0.200181021447


In [1]:
print(test_spectra)

NameError: name 'test_spectra' is not defined

In [2]:
import feather

ImportError: No module named 'feather'